# Experiment 1: Model Validation

### Description

Plot key system variables and validate that the model is working as expected.

### Assumptions

### Experiment Setup

There are several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

In [62]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pprint import pprint

import visualizations
from experiments.run import run
from experiments.utils import display_code

time: 24.2 ms (started: 2021-09-16 12:04:25 +02:00)


In [63]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = True

time: 21.4 ms (started: 2021-09-16 12:04:25 +02:00)


In [64]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

time: 21.6 ms (started: 2021-09-16 12:04:25 +02:00)


In [82]:
# Inspect experiment template:
#display_code(time_domain_analysis)


time: 20.7 ms (started: 2021-09-16 12:07:55 +02:00)


In [66]:
# Create a new copy of the relevant simulation for each analysis
simulation_1 = copy.deepcopy(time_domain_analysis.experiment.simulations[0])

time: 30.5 ms (started: 2021-09-16 12:04:25 +02:00)


# Adjust experiment

### Override Initial State

In [68]:
# Check initial state:

pprint(simulation_1.model.initial_state)

{'avg_price': 2,
 'clients': 5,
 'hosts': 1,
 'hosts_daily_profit': 0,
 'hosts_daily_revenue': 0,
 'indicated_network_demand': 0,
 'network_allocation': 0,
 'network_capacity': 1,
 'network_penetration': 0,
 'platform_daily_revenue': 0,
 'potential_users': 10000}
time: 29.8 ms (started: 2021-09-16 12:04:25 +02:00)


In [69]:
simulation_1.model.initial_state.update({
    "hosts": 1, 
    "clients": 1,
    "avg_price": 4
})

time: 29.9 ms (started: 2021-09-16 12:04:25 +02:00)


### Override parameters

In [70]:
pprint(simulation_1.model.params)

{'MIN_expected_fulfillment': [0.33],
 'avg_client_allocation': [10],
 'avg_host_line': [1000],
 'avg_reserve_capacity': [0.25],
 'client_competitor_price': [2],
 'client_registration_delay': [14],
 'host_line_cost': [0.05],
 'host_setup_delay': [60],
 'host_technical_difficulty': [0.5],
 'initial_population': [10000],
 'network_inefficiencies': [0.2],
 'onboarding_coefficient': [0.01],
 'price_change_delay': [14],
 'service_fee': [0.025]}
time: 22.3 ms (started: 2021-09-16 12:04:25 +02:00)


In [71]:
simulation_1.model.params.update({
    "host_setup_delay": [90],
    "client_registration_delay": [14],
    "MIN_expected_fulfillment": [0.33],
    "onboarding_coefficient": [1],
    "price_change_delay": [14],
    "avg_host_line": [1000],
    "service_fee": [0.05]
})

time: 43.2 ms (started: 2021-09-16 12:04:25 +02:00)


In [72]:
# Execute the simulation (radCAD):

df_1, _exceptions = run(simulation_1)

time: 95.7 ms (started: 2021-09-16 12:04:25 +02:00)


In [73]:
# View DataFrame:
#df_1.head(100)

time: 27 ms (started: 2021-09-16 12:04:25 +02:00)


# Anaysis 1: User Adoption

Plot user adoption over the time horizon.

The rate of user adoption at any given timestep is a function of: 
- **word-of-mouth** (the more clients there are, the more likely )
- **base onboarding rate** (set in system parameters as 'onboarding_coeff')
- **price attractiveness** for the given agent (i.e. lower price = more attractive to clients but less attractive for hosts) 
- **registration delay** (i.e the average time clients take to register)

In [74]:
# fig_df = df_1.query('variable_name == 2')

fig = px.line(
    df_1,
    x='timestep',
    y=['hosts', 'clients'],
    height=300
)

fig.update_layout(
    title="User Adoption",
    xaxis_title="Days",
    yaxis_title="Persons",
    legend_title="User Type")

fig.show()

time: 90.6 ms (started: 2021-09-16 12:04:25 +02:00)


# Anaysis 2: Network Allocation

Plot the actual bandwidth allocation for the network over time.

Notes: 
- Since the network assumes clients can initiate temporary connections, actual network allocation is dependent on the current price attractiveness to existing clients, v.s. the 

In [75]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_allocation'],
    height=350
)

fig.update_layout(
    title="Network Allocation",
    xaxis_title="Days",
    yaxis_title="Mbps",
    legend_title="Allocation")

fig.show()

time: 85 ms (started: 2021-09-16 12:04:25 +02:00)


In [76]:
# Use Pandas to query results 

#E.g. maximum network allocation
df_1.groupby('subset')['network_allocation'].max()

subset
0    449347.908974
1    448305.079595
Name: network_allocation, dtype: float64

time: 29.7 ms (started: 2021-09-16 12:04:25 +02:00)


In [77]:
fig = px.line(
    df_1,
    x='timestep',
    y=['avg_price'],
    height=300
)

fig.update_layout(
    title="AVG Host Price over Time",
    xaxis_title="Days",
    yaxis_title="ZAR/Mbps/Day",
    legend_title="Price")

fig.show()

time: 76.4 ms (started: 2021-09-16 12:04:25 +02:00)


In [78]:
# Plot 3D Surface:

# visualizations.plot_surface(df_1)

time: 30.9 ms (started: 2021-09-16 12:04:25 +02:00)


In [79]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_penetration'],
    height=300
)

fig.update_layout(
    title="Network Penetration",
    xaxis_title="Days",
    yaxis_title="%",
    legend_title="%")

fig.show()

time: 77.3 ms (started: 2021-09-16 12:04:25 +02:00)


# Anaysis: Yields

Host and platform yeilds

In [80]:
fig = px.line(
    df_1,
    x='timestep',
    y=['hosts_daily_revenue', 'hosts_daily_profit', 'platform_daily_revenue'],
    height=400
)

fig.update_layout(
    title="Daily Revenue & Profit Yields",
    xaxis_title="Days",
    yaxis_title="ZAR/Day",
    legend_title="Agents"
)

newnames = {'hosts_daily_revenue':'Hosts (revenue)', 'hosts_daily_profit': 'Hosts (profit)', 'platform_daily_revenue': 'Platform (revenue)'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
        
fig.show()

time: 96.7 ms (started: 2021-09-16 12:04:26 +02:00)


# Analysis: Hosts & Platform Yields

In [81]:
# Simulation metrics
period = df_1['timestep'].max()
service_fee = simulation_1.model.params["service_fee"][0]

# Pandas calculations:
host_cumulative_revenue = int(df_1['hosts_daily_revenue'].sum())
host_cumulative_profit = int(df_1['hosts_daily_profit'].sum())
platform_cumulative_revenue = int(df_1['platform_daily_revenue'].sum())

# Output formatting:
service_fee_formatted = service_fee * 100
revenue_formatted = "R{:,.0f}".format(host_cumulative_revenue)
profit_formatted = "R{:,.0f}".format(host_cumulative_profit)
platform_revenue_formatted = "R{:,.0f}".format(platform_cumulative_revenue)

# Display output:
print("Host cumulative revenue (", period , "days ):")
print(revenue_formatted, "\n")

print("Host cumulative profit (", period , "days ):")
print(profit_formatted, "\n")

print("Platform cumulative revenue (", service_fee_formatted, "% fee) (", period , "days ):")
print(platform_revenue_formatted, "\n")

Host cumulative revenue ( 365 days ):
R126,157,498 

Host cumulative profit ( 365 days ):
R99,450,876 

Platform cumulative revenue ( 5.0 % fee) ( 365 days ):
R6,270,860 

time: 27.4 ms (started: 2021-09-16 12:04:26 +02:00)
